# 14. 멀리 있는 사람도 스티커를 붙여주자

# 14-1. 들어가며

### WIDER FACE 데이터셋
face detection 모델의 학습을 위해 [WIDER FACE 데이터셋](http://shuoyang1213.me/WIDERFACE/index.html)을 다룰 예정이다.   
빠른 YOLO, SSD같은 single stage model을 학습시키는 것은 흔히 COCO 데이터셋이 사용되겠지만, 먼 거리에 흩어져있는 여러 사람의 얼굴을 빠르게 detect하는 모델을 만들기 위해 보다 넓은 공간에서 다수의 사람이 등장하는 이미지 데이터셋이 적합하다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-11-P-00.max-800x600.png)

### 데이터셋 준비하기
WIDER FACE 데이터셋 홈페이지에 있는 4개의 zip파일을 사용할 예정이다.

홈페이지의 Download에 해당한다.
- [WIDER Face Training Images Google Drive](https://drive.google.com/file/d/15hGDLhsx8bLgLcIRD5DhYt5iBxnjNF1M/view)
- [WIDER Face Validation Images Google Drive](https://drive.google.com/file/d/1GUCogbp16PMGa39thoMMeWxp7Rp5oM8Q/view)
- [WIDER Face Testing Images Google Drive](https://drive.google.com/file/d/1HIfDbVEWKmsYKJZm4lchTBDLW5N7dY5T/view)
- [Face annotations WIDER FACE 데이터셋 홈페이지](http://shuoyang1213.me/WIDERFACE/index.html)

위 파일들을 데이터셋 PATH에 widerface 폴더를 만들어서 unzip 해주자. 그리고 아이펠에서 제공한 9개의 python 모듈 파일이 포함되어 있는 파일도 PATH에서 unzip 하자.

`unzip '*.zip'`

# 14-2. 데이터셋 전처리(1) 분석

### WIDER FACE Bounding Box
오늘 다룰 WIDER FACE 데이터셋은 face detection을 위한 데이터셋으로, 입력 데이터는 이미지 파일로, Ground truth는 bounding box 정보로 되어있다.

In [3]:
!cd /content/drive/MyDrive/aiffel_dataset/GD14_face_sticker/widerface && ls *

wider_face_split.zip  WIDER_test.zip  WIDER_train.zip  WIDER_val.zip

wider_face_split:
readme.txt		      wider_face_train_bbx_gt.txt  wider_face_val.mat
wider_face_test_filelist.txt  wider_face_train.mat
wider_face_test.mat	      wider_face_val_bbx_gt.txt

WIDER_test:
images

WIDER_train:
images

WIDER_val:
images


WIDER_test, train, val 디렉토리는 입력용 이미지 파일만 들어와 있다.   
wider_face_split 디랙토리 내에 wider_face_train_bbx_gt.txt와 wider_face_val_bbx_gt.txt 파일 안에 포함되어 있는 bounding box 정보를 알아보자.

In [4]:
!cd /content/drive/MyDrive/aiffel_dataset/GD14_face_sticker/widerface/wider_face_split/ && head -20 wider_face_train_bbx_gt.txt

0--Parade/0_Parade_marchingband_1_849.jpg
1
449 330 122 149 0 0 0 0 0 0 
0--Parade/0_Parade_Parade_0_904.jpg
1
361 98 263 339 0 0 0 0 0 0 
0--Parade/0_Parade_marchingband_1_799.jpg
21
78 221 7 8 2 0 0 0 0 0 
78 238 14 17 2 0 0 0 0 0 
113 212 11 15 2 0 0 0 0 0 
134 260 15 15 2 0 0 0 0 0 
163 250 14 17 2 0 0 0 0 0 
201 218 10 12 2 0 0 0 0 0 
182 266 15 17 2 0 0 0 0 0 
245 279 18 15 2 0 0 0 0 0 
304 265 16 17 2 0 0 0 2 1 
328 295 16 20 2 0 0 0 0 0 
389 281 17 19 2 0 0 0 2 0 
406 293 21 21 2 0 1 0 0 0 


이는 다음과 같은 반복 구조로 이루어져 있음을 파악할 수 있다.

```
# 이미지 파일 경로
0--Parade/0_Parade_marchingband_1_849.jpg
# face bounding box 개수
1
# face bounding box 좌표 등 상세정보
449 330 122 149 0 0 0 0 0 0 
```

10개의 숫자로 이루어진 정보들은 다음과 같은 상세정보를 갖고 있다.   
```
x0, y0, w, h, blur, expression, illumination, invalid, occlusion, pose
```

bounding box와 관련해서 가장 중요한 4개의 숫자는 왼쪽의 4개(X좌표, Y좌표, 너비, 높이)이다.

In [5]:
import os, cv2, time
import tensorflow as tf
import tqdm
import numpy as np
import math
from itertools import product
import matplotlib.pyplot as plt

PROJECT_PATH = '/content/drive/MyDrive/aiffel_dataset/GD14_face_sticker'
DATA_PATH = os.path.join(PROJECT_PATH, 'widerface')
MODEL_PATH = os.path.join(PROJECT_PATH, 'checkpoints')
TRAIN_TFRECORD_PATH = os.path.join(PROJECT_PATH, 'dataset', 'train_mask.tfrecord')
VALID_TFRECORD_PATH = os.path.join(PROJECT_PATH, 'dataset', 'val_mask.tfrecord')
CHECKPOINT_PATH = os.path.join(PROJECT_PATH, 'checkpoints')

DATASET_LEN = 12880
BATCH_SIZE = 32
IMAGE_WIDTH = 320
IMAGE_HEIGHT = 256
IMAGE_LABELS = ['background', 'face']

print(PROJECT_PATH)
print(tf.__version__)

/content/drive/MyDrive/aiffel_dataset/GD14_face_sticker
2.8.0


먼저 bounding box 파일을 분석해보자.

In [ ]:
def parse_box(data):
    x0 = int(data[0])
    y0 = int(data[1])
    w = int(data[2])
    h = int(data[3])
    return x0, y0, w, h

print('슝=3')

In [ ]:
def parse_widerface(file):
    infos = []
    with open(file) as fp:
        line = fp.readline()
        while line:
            n_object = int(fp.readline())
            boxes = []
            for i in range(n_object):
                box = fp.readline().split(' ')
                x0, y0, w, h = parse_box(box)
                if (w == 0) or (h == 0):
                    continue
                boxes.append([x0, y0, w, h])
            if n_object == 0:
                box = fp.readline().split(' ')
                x0, y0, w, h = parse_box(box)
                boxes.append([x0, y0, w, h])
            infos.append((line.strip(), boxes))
            line = fp.readline()
    return infos

print('슝=3')

위 함수는 이미지별 bounding box 정보를 `wider_face_train_bbx_gt.txt`에서 파싱해서 리스트로 추출하는 것이다.

추출한 정보를 실제 이미지 정보와 결합해보자.   
bounding box 정보는 [x, y, w, h] 형태로 저장되어 있는데, [x_min, y_min, x_max, y_max] 형태의 꼭짓점 좌표 정보로 변환해야한다.

이렇게 정보를 결합해야 나중에 학습에 용이하다.

In [ ]:
def process_image(image_file):
    image_string = tf.io.read_file(image_file)
    try:
        image_data = tf.image.decode_jpeg(image_string, channels=3)
        return 0, image_string, image_data
    except tf.errors.InvalidArgumentError:
        return 1, image_string, None

print('슝=3')

In [ ]:
def xywh_to_voc(file_name, boxes, image_data):
    shape = image_data.shape
    image_info = {}
    image_info['filename'] = file_name
    image_info['width'] = shape[1]
    image_info['height'] = shape[0]
    image_info['depth'] = 3

    difficult = []
    classes = []
    xmin, ymin, xmax, ymax = [], [], [], []

    for box in boxes:
        classes.append(1)
        difficult.append(0)
        xmin.append(box[0])
        ymin.append(box[1])
        xmax.append(box[0] + box[2])
        ymax.append(box[1] + box[3])
    image_info['class'] = classes
    image_info['xmin'] = xmin
    image_info['ymin'] = ymin
    image_info['xmax'] = xmax
    image_info['ymax'] = ymax
    image_info['difficult'] = difficult

    return image_info

print('슝=3')

이렇게 결합된 데이터의 형태를 5개만 확인해보자.   
그리고 이 정보를 활용하여 텐서플로우 데이터셋을 생성해보자.

In [ ]:
file_path = os.path.join(DATA_PATH, 'wider_face_split', 'wider_face_train_bbx_gt.txt')
for i, info in enumerate(parse_widerface(file_path)):
    print('--------------------')
    image_file = os.path.join(DATA_PATH, 'WIDER_train', 'images', info[0])
    _, image_string, image_data = process_image(image_file)
    boxes = xywh_to_voc(image_file, info[1], image_data)
    print(boxes)
    if i > 3:
        break

# 14-3. 데이터셋 전처리(2) TFRecord 생성

### TFRecord 만들기
오늘 다루는 대용량 데이터셋의 처리속도 향상을 위해, 전처리 작업을 통해 TFRecord 데이터셋으로 변환해보자.   
TFRecord란 TensorFlow만의 학습 데이터 저장 포맷으로, 이진 레코드의 스퀀스를 저장한다. TFRecord 형태의 학습 데이터를 사용하여 모델 학습을 하면 학습 속도가 개선된다.

TFRecord는 여러 개의 tf.train.Example로 이루어져 있고, 한 개의 tf.train.Example은 여러 개의 tf.train.Feature로 이루어져 있다.

데이터 단위를 이루는 tf.train.Example 인스턴스를 생성하는 메소드를 선언해보자.

In [ ]:
def make_example(image_string, image_infos):
    for info in image_infos:
        filename = info['filename']
        width = info['width']
        height = info['height']
        depth = info['depth']
        classes = info['class']
        xmin = info['xmin']
        ymin = info['ymin']
        xmax = info['xmax']
        ymax = info['ymax']

    if isinstance(image_string, type(tf.constant(0))):
        encoded_image = [image_string.numpy()]
    else:
        encoded_image = [image_string]

    base_name = [tf.compat.as_bytes(os.path.basename(filename))]
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'filename':tf.train.Feature(bytes_list=tf.train.BytesList(value=base_name)),
        'height':tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'width':tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'classes':tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        'x_mins':tf.train.Feature(float_list=tf.train.FloatList(value=xmin)),
        'y_mins':tf.train.Feature(float_list=tf.train.FloatList(value=ymin)),
        'x_maxes':tf.train.Feature(float_list=tf.train.FloatList(value=xmax)),
        'y_maxes':tf.train.Feature(float_list=tf.train.FloatList(value=ymax)),
        'image_raw':tf.train.Feature(bytes_list=tf.train.BytesList(value=encoded_image))
    }))
    
    return example

print('슝=3')

전처리에 필요한 함수들이 갖추어졌다.   
데이터셋의 이미지 파일, 그리고 bounding box 정보를 모아 위의 make_example 메소드를 통해 만든 example을 serialize하여 TFRecord 파일로 생성하게 된다.
- [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord)

In [ ]:
for split in ['train', 'val']:
    if split == 'train':
        output_file = TRAIN_TFRECORD_PATH 
        anno_txt = 'wider_face_train_bbx_gt.txt'
        file_path = 'WIDER_train'
    else:
        output_file = VALID_TFRECORD_PATH
        anno_txt = 'wider_face_val_bbx_gt.txt'
        file_path = 'WIDER_val'

    with tf.io.TFRecordWriter(output_file) as writer:
        for info in tqdm.tqdm(parse_widerface(os.path.join(DATA_PATH, 'wider_face_split', anno_txt))):
            image_file = os.path.join(DATA_PATH, file_path, 'images', info[0])
            error, image_string, image_data = process_image(image_file)
            boxes = xywh_to_voc(image_file, info[1], image_data)

            if not error:
                tf_example = make_example(image_string, [boxes])
                writer.write(tf_example.SerializeToString())

In [ ]:
!ls /content/drive/MyDrive/aiffel_dataset/GD14_face_sticker/dataset

# 14-4. 모델 구현(1) Default boxes

### SSD의 Default box
SSD 모델의 가장 중요한 특징 중 하나는 Default box를 필요로 한다는 점이다.   
Default box란, object가 존재할만한 다양한 크기의 box 좌표 및 클래스 정보를 일정 개수만큼 미리 고정해 둔 것이다. 이를 anchor box, prior box라고 부른다.   
SSD의 default box의 다른 점은 여러 층의 feature map에서 box를 만든다는 점이다. 층 수 만큼 box도 많아지고, 층마다 box의 크기도 다양해진다.   
ground truth에 해당하는 bounding box의 IoU를 계산하여 일정 크기(0.5) 이상 겹치는 default box를 선택하는 방식이 RCNN 계열의 sliding window 방식보다 훨씬 속도가 빠르면서 유사한 정확도를 얻을 수 있다.   
![](https://d3s0tskafalll9.cloudfront.net/media/images/gc-9v3-p-4-1_adjeL1p.max-800x600.jpg)

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-11-P-06.max-800x600.png)

default box를 생성하기 위해 기준이 되는 feature map을 먼저 생성해야한다. 그림에서는 8x8, 4x4의 예가 나오지만 이번에는 4가지 유형의 feature map을 생성해보자.

In [ ]:
BOX_MIN_SIZES = [[10, 16, 24], [32, 48], [64, 96], [128, 192, 256]]
BOX_STEPS = [8, 16, 32, 64]

print('슝=3')

In [ ]:
image_sizes = (IMAGE_HEIGHT, IMAGE_WIDTH)
min_sizes = BOX_MIN_SIZES
steps= BOX_STEPS

feature_maps = [
    [math.ceil(image_sizes[0] / step), math.ceil(image_sizes[1] / step)]
    for step in steps
]
feature_maps

feature map별로 순회를 하면서 default box를 생성해보자.

In [ ]:
boxes = []
for k, f in enumerate(feature_maps):
    for i, j in product(range(f[0]), range(f[1])):
        for min_size in min_sizes[k]:
            s_kx = min_size / image_sizes[1]
            s_ky = min_size / image_sizes[0]
            cx = (j + 0.5) * steps[k] / image_sizes[1]
            cy = (i + 0.5) * steps[k] / image_sizes[0]
            boxes += [cx, cy, s_kx, s_ky]

len(boxes)

이렇게 생성된 boxes는 default box 정보가 구분없이 나열되어 있으므로 4개씩 재배열해주자.

In [ ]:
pretty_boxes = np.asarray(boxes).reshape([-1, 4])
print(pretty_boxes.shape)
print(pretty_boxes)

4700개의 default box가 만들어졌다. feature_maps와 min_size로부터 40x32x3 + 20x16x2 + 10x8x2 + 5x4x3 개가 생성됐다.

위 과정을 사용하기 편리하도록 함수로 정의해보자.

In [ ]:
def default_box():
    image_sizes = (IMAGE_HEIGHT, IMAGE_WIDTH)
    min_sizes = BOX_MIN_SIZES
    steps= BOX_STEPS
    feature_maps = [
        [math.ceil(image_sizes[0] / step), math.ceil(image_sizes[1] / step)]
        for step in steps
    ]
    boxes = []
    for k, f in enumerate(feature_maps):
        for i, j in product(range(f[0]), range(f[1])):
            for min_size in min_sizes[k]:
                s_kx = min_size / image_sizes[1]
                s_ky = min_size / image_sizes[0]
                cx = (j + 0.5) * steps[k] / image_sizes[1]
                cy = (i + 0.5) * steps[k] / image_sizes[0]
                boxes += [cx, cy, s_kx, s_ky]
    boxes = np.asarray(boxes).reshape([-1, 4])
    return boxes

print('슝=3')

# 14-5. 모델 구현(2) SSD

### SSD model 빌드하기
SSD 모델을 생성해보자.

SSD 모델에서 일반적으로 많이 쓰이는 convolution block, depthwise convolution block, skip connection으로 쓰일 branch block을 준비하자.

In [ ]:
def _conv_block(inputs, filters, kernel=(3, 3), strides=(1, 1)):
    block_id = (tf.keras.backend.get_uid())
    if strides == (2, 2):
        x = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name='conv_pad_%d' % block_id)(inputs)
        x = tf.keras.layers.Conv2D(filters, kernel,
                                   padding='valid',
                                   use_bias=False,
                                   strides=strides,
                                   name='conv_%d' % block_id)(x)
    else:
        x = tf.keras.layers.Conv2D(filters, kernel,
                                   padding='same',
                                   use_bias=False,
                                   strides=strides,
                                   name='conv_%d' % block_id)(inputs)
    
    x = tf.keras.layers.BatchNormalization(name='conv_bn_%d' % block_id)(x)
    return tf.keras.layers.ReLU(name='conv_relu_%d' % block_id)(x)

print('슝=3')

In [ ]:
def _depthwise_conv_block(inputs, filters, strides=(1, 1)):
    block_id = tf.keras.backend.get_uid()
    if strides == (1, 1):
        x = inputs
    else:
        x = tf.keras.layers.ZeroPadding2D(((1, 1), (1, 1)), name='conv_pad_%d' % block_id)(inputs)
    x = tf.keras.layers.DepthwiseConv2D((3, 3),
                                        padding='same' if strides == (1, 1) else 'valid',
                                        strides=strides,
                                        use_bias=False,
                                        name='conv_dw_%d' % block_id)(x)
    x = tf.keras.layers.BatchNormalization(name='conv_dw_%d_bn' % block_id)(x)
    x = tf.keras.layers.ReLU(name='conv_dw_%d_relu' % block_id)(x)
    x = tf.keras.layers.Conv2D(filters, (1, 1),
                               padding='same',
                               use_bias=False,
                               strides=(1, 1),
                               name='conv_pw_%d' % block_id)(x)
    x = tf.keras.layers.BatchNormalization(name='conv_pw_%d_bn' % block_id)(x)
    return tf.keras.layers.ReLU(name='conv_pw_%d_relu' % block_id)(x)

print('슝=3')

In [ ]:
def _branch_block(inputs, filters):
    x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(inputs)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x1 = tf.keras.layers.Conv2D(filters * 2, kernel_size=(3, 3), padding='same')(inputs)
    x = tf.keras.layers.Concatenate(axis=-1)([x, x1])
    return tf.keras.layers.ReLU()(x)

print('슝=3')

여러 블록을 쌓아 모델을 만든 후, 중간중간 Branch 부분에 head라고 불리는 convolution layer를 붙일것이다.   
하나의 head에 convolution layer 2개가 필요하다. 하나는 confidence를 예측하기 위해, 하나는 location을 예측하기 위해 사용한다.

branch마다 head가 연결되어 있기 때문에 모델의 중간 레이어에서도 예측을 위한 정보를 가져올 수 있다.

In [ ]:
def _create_head_block(inputs, filters):
    x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), strides=(1, 1), padding='same')(inputs)
    return x

print('슝=3')

In [ ]:
def _compute_heads(inputs, num_class, num_cell):
    conf = _create_head_block(inputs, num_cell * num_class)
    conf = tf.keras.layers.Reshape((-1, num_class))(conf)
    loc = _create_head_block(inputs, num_cell * 4)
    loc = tf.keras.layers.Reshape((-1, 4))(loc)
    return conf, loc

print('슝=3')

# 14-6. 모델 학습(1) Augmentation, jaccard 적용

# 14-7. 모델 학습(2) train

# 14-8. inference(1) NMS

# 14-9. inference(2) 사진에서 얼굴 찾기

# 14-10. 프로젝트: 스티커를 붙여주자